In [37]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
import nltk

nltk.download()

True

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\s+', gaps=True)

tokenizer.tokenize("As-Salam-o-Alikum Sir, the name of the tutorial is \ Installation of NLTK package and downloading of NLTK \ corpus on Python")



In [9]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [10]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### 3. Limpieza y procesamiento del los tweets

In [38]:
# Eliminaremos los artículos, proposiciones (stopwords) y otras palabras que no aportan un valor conceptual en nuestros tweets

def limpiar_tokenizar(texto):
    '''
    Esta función limpia y tokeniza el texto en palabras individuales.
    El orden en el que se va limpiando el texto no es arbitrario.
    El listado de signos de puntuación se ha obtenido de: print(string.punctuation)
    y re.escape(string.punctuation)
    '''
    
    # Se convierte todo el texto a minúsculas
    nuevo_texto = texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 1]
    
    return(nuevo_texto)


# Eliminando las stopwords de inglés

# Encontramos el listado de stopwords en inglés
stop_words = list(stopwords.words('english'))
# Se añade la stoprword: amp, ax, ex
stop_words.extend(("amp", "xa", "xe"))
print(stop_words[:10])


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\carlo/nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\carlo\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\carlo\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [27]:
# Realizamos la limpieza para cada uno de los tweets

train['texto_tokenizado'] = train['text'].apply(lambda x: limpiar_tokenizar(x))
train.head()

,id,keyword,location,text,target,texto_tokenizado
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala..."


In [32]:
# Ahora por cada token se tendra una fila (con mismo id, keyword, location y target), con el objetivo de facilitar 
# la exploración de data

# Unnest de la columna texto_tokenizado
# ==============================================================================
tweets_tidy = train.explode(column='texto_tokenizado')
tweets_tidy = tweets_tidy.drop(columns='text')
tweets_tidy = tweets_tidy.rename(columns={'texto_tokenizado':'token'})
tweets_tidy.head()

,id,keyword,location,target,token
0,1,NaN,NaN,1,our
0,1,NaN,NaN,1,deeds
0,1,NaN,NaN,1,are
0,1,NaN,NaN,1,the
0,1,NaN,NaN,1,reason


### 4. Frecuencia 

In [33]:
# Total de palabras utilizadas en tweets de desastre y no desastre
# ==============================================================================
print('--------------------------')
print('Palabras por real desastre (1) o no (0) ')
print('--------------------------')
tweets_tidy.groupby(by='target')['token'].count()

--------------------------
Palabras por real desastre (1) o no (0) 
--------------------------


target
0    57628
1    44380
Name: token, dtype: int64

Se tienen 13 mil palabras más provenientes de tweets con desastres reales.

#### Palabras más utilizadas en tweets desastres reales o no

In [34]:
# Encontramos las 10 palabras más utilizadas en los tipos de tweets

tweets_tidy.groupby(['target','token'])['token'] \
 .count() \
 .reset_index(name='count') \
 .groupby('target') \
 .apply(lambda x: x.sort_values('count', ascending=False).head(5))

target token  count
target                           
0      10394       0   the   1914
       10583       0    to   1191
       351         0   and    919
       7282        0    of    902
       5084        0    in    822
1      19486       1   the   1363
       15655       1    in   1162
       17241       1    of    923
       19627       1    to    757
       12168       1   and    505